In [1]:
import numpy as np
import os
import pandas as pd
import geopandas as gpd
import glob

In [2]:
import os
import pandas as pd
import geopandas as gpd

# Define directories
data_dir = r"C:\Users\betebari\Documents\briny-injection-AqExemptions\post 2018"
output_dir = os.path.join(data_dir, "merged")
os.makedirs(output_dir, exist_ok=True)

# Load allwells data
allwells = pd.read_csv("AllWellsCV.csv", dtype={'API': str})

# Define years for processing
years = range(2018, 2022)  # Adjust the range for the years you want to process

def read_file(file_path):
    """Helper function to read either CSV or XLS files."""
    if file_path.endswith('.csv'):
        return pd.read_csv(file_path, dtype={'APINumber': str})
    elif file_path.endswith('.xls') or file_path.endswith('.xlsx'):
        return pd.read_excel(file_path, dtype={'APINumber': str})
    else:
        raise ValueError(f"Unsupported file format: {file_path}")

# Process injection data
for year in years:
    try:
        # Dynamically find file path for injection data
        inj_file = glob.glob(os.path.join(data_dir, f"{year}*injection*"))[0]
        inj_data = read_file(inj_file)

        inj_data.APINumber = inj_data.APINumber.str[2:-2]  # Adjust API format
        inj_data['Gas/AirInjected(MCF)'] = inj_data.get('GasAirInjected', None)
        inj_data['Steam/WaterInjected(BBL)'] = inj_data.get('SteamWaterInjected', None)
        inj_data['year'] = year

        print(f'inj{year}', inj_data.shape[0])
        
        # Merge with allwells
        inj_data = pd.merge(inj_data, allwells, left_on='APINumber', right_on='API', how='left')
        print(f'inj{year}', inj_data.shape[0])
        
        # Convert to GeoDataFrame
        inj_gdf = gpd.GeoDataFrame(
            inj_data,
            geometry=gpd.points_from_xy(inj_data.Longitude, inj_data.Latitude)
        ).set_crs('epsg:4269').to_crs('epsg:3310')

        # Save the processed data
        inj_gdf.to_csv(os.path.join(output_dir, f"injection_{year}.csv"), index=False)
        print(f"Injection data for {year} processed and saved.")
    except Exception as e:
        print(f"Error processing injection data for {year}: {e}")

# Process production data
for year in years:
    try:
        # Dynamically find file path for production data
        prod_file = glob.glob(os.path.join(data_dir, f"{year}*production*"))[0]
        prod_data = read_file(prod_file)

        prod_data.APINumber = prod_data.APINumber.str[2:-2]  # Adjust API format
        prod_data['GasProduced(MCF)'] = prod_data.get('GasProduced', None)
        prod_data['WaterProduced(BBL)'] = prod_data.get('WaterProduced', None)
        prod_data['ProductionDate'] = prod_data.get('ProductionReportDate', None)
        prod_data['year'] = year

        print(f'prod{year}', prod_data.shape[0])
        
        # Merge with allwells
        prod_data = pd.merge(prod_data, allwells, left_on='APINumber', right_on='API', how='left')
        print(f'prod{year}', prod_data.shape[0])
        
        # Convert to GeoDataFrame
        prod_gdf = gpd.GeoDataFrame(
            prod_data,
            geometry=gpd.points_from_xy(prod_data.Longitude, prod_data.Latitude)
        ).set_crs('epsg:4269').to_crs('epsg:3310')

        # Save the processed data
        prod_gdf.to_csv(os.path.join(output_dir, f"production_{year}.csv"), index=False)
        print(f"Production data for {year} processed and saved.")
    except Exception as e:
        print(f"Error processing production data for {year}: {e}")


inj2018 498535
inj2018 498535
Injection data for 2018 processed and saved.
inj2019 515338
inj2019 515338
Injection data for 2019 processed and saved.
Error processing injection data for 2020: Excel file format cannot be determined, you must specify an engine manually.
Error processing injection data for 2021: Excel file format cannot be determined, you must specify an engine manually.
prod2018 1017165
prod2018 1017165
Production data for 2018 processed and saved.
prod2019 1042623
prod2019 1042623
Production data for 2019 processed and saved.
Error processing production data for 2020: Excel file format cannot be determined, you must specify an engine manually.
Error processing production data for 2021: Excel file format cannot be determined, you must specify an engine manually.
